# Treating acute mountain sickness

In [1]:
# Our usual imports
import numpy as np
# Make random number generator.
rng = np.random.default_rng()
import pandas as pd
# Safe setting for Pandas.  Needs Pandas version >= 1.5.
pd.set_option('mode.copy_on_write', True)

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

# The OKpy testing system.
from client.api.notebook import Notebook
ok = Notebook('acetazolamide.ok')

Here we are looking at some data from this experiment:

> Emma V. Low et al. "Identifying the lowest effective dose of
acetazolamide for the prophylaxis of acute mountain sickness: systematic review
and meta-analysis." BMJ, 345, :doi:`10.1136/bmj.e6779`, 2012.

The paper studies doses of the drug *acetazolamide* in treating *acute mountain
sickness*.  Acute mountain sickness (AMS) is a mild form of the potentially
fatal [altitude sickness](https://en.wikipedia.org/wiki/Altitude_sickness),
caused by "rapid exposure to low amounts of oxygen at high elevation".
Symptoms are nausea, headaches, dizziness and fatigue.

The paper concludes:

> Acetazolamide 250 mg, 500 mg, and 750 mg daily were all efficacious for
preventing acute mountain sickness. Acetazolamide 250 mg was the lowest
effective dose with available evidence for this indication.

We can reconstruct a table from their data that shows the effect of 250mg of
acetazolamide daily in preventing AMS, compared to placebo.

In [2]:
# Build AMS / Drug contingency DataFrame
counts = [[7, 17], [15, 5]]  #  A list of lists
ams_counts = pd.DataFrame(counts,
                          # Set the column labels.
                          columns=['Acetazolamide', 'Placebo'],
                          # Set the row labels.
                          index=['Had AMS', 'No AMS'])
ams_counts

This is a *counts* table, giving the number of people in each category.

For example, here is the number of people who did get AMS, who were in the
250mg daily acetazolamide group:

In [3]:
ams_counts.loc['Had AMS', 'Acetazolamide']

For our usual task of *inference*, this table gives the *data*.

How convincing is the evidence in this table, that 250mg of acetazolamide is
effective in treating AMS?

We begin by formulating a null model.  Remember, the null model is a model that
is as close as possible to the real world, but with the effect we are
interested in set to 0.

> The odds of experiencing AMS after treatment with acetazolamide are *not
> different from* (are the same as) the odds of experiencing AMS after placebo.

We first identify the *statistic* we are going to use.  We use the number of
people who did have AMS, who also took the Acetazolamide.

In [4]:
observed_stat = ams_counts.loc['Had AMS', 'Acetazolamide']
observed_stat

Our next step is to simulate one trial in the null-world, in which the
association is random between AMS (Has and No) and drug (Acetazolamide and
Placebo).

In order to do that, we want to:

* reconstruct the original *Observations* DataFrame from the Counts DataFrame
  that we have.  The Observations DataFrame has one row per person, and the
  columns are "AMS" (Yes or No) and "Drug" (Acetazolamide or Placebo).
* Simulate a random association, by shuffling the AMS (or Drug) column.
* Regenerate the cross-tabulation for this shuffled DataFrame.
* Get the statistic of interest (the "Has AMS", "Acetazolamide" count).

The most involved part of this process is first step, going from Counts to
Observations.

Go back to the [noble politics page](https://lisds.github.io/textbook/wild-pandas/noble_politics) for a refresher on this process.

The total number of observations in the Counts data frame is:

In [5]:
# Sum of the row totals.
n_total = ams_counts.sum().sum()
n_total

First make a list of lists, where each list defines one row, and there is one
list for every row in the Observations DataFrame.  Call this list of lists
`row_lists`.

For example, the first three lists in `row_lists` will be:

In [6]:
first_three = [['Had AMS', 'Acetazolamide'],
               ['Had AMS', 'Acetazolamide'],
               ['Had AMS', 'Acetazolamide']]
first_three

The last three lists in `row_lists` will be:

In [7]:
last_three = [['No AMS', 'Placebo'],
              ['No AMS', 'Placebo'],
              ['No AMS', 'Placebo']]
last_three

We can get the same list of lists by *multiplying* the list:

In [8]:
[['No AMS', 'Placebo']] * 3

If you are stuck, check back to the Noble Politics page.

In [9]:
row_lists = (  # Parentheses just to allow you to split the next
               # statement across many lines, if you want.
    ...
)
row_lists = (
    [['Had AMS', 'Acetazolamide']] * ams_counts.loc['Had AMS', 'Acetazolamide'] +
    [['Had AMS', 'Placebo']] * ams_counts.loc['Had AMS', 'Placebo'] + 
    [['No AMS', 'Acetazolamide']] * ams_counts.loc['No AMS', 'Acetazolamide'] +
    [['No AMS', 'Placebo']] * ams_counts.loc['No AMS', 'Placebo']
)
# Show the last five lists
row_lists[-5:]

In [10]:
_ = ok.grade('q_01_row_lists')

Use `row_lists` to create an Observations DataFrame `people`, with one row per person in the study.  The first column of the DataFrame should contain the AMS label ('Had AMS' or 'No AMS'), and have the column name `AMS`.  The second column should contain the drug label (`Acetazolamide` or `Placebo`) and be called `Drug` (notice the capitalization).

In [11]:
people = ...
people = pd.DataFrame(row_lists,
                      columns=['AMS', 'Drug'])
# Show the result.
people

In [12]:
_ = ok.grade('q_02_people')

Extract the `AMS` column of `people` as the variable `ams`.

Extract the `Drug` column of `people` as the variable `drug`.

In [13]:
ams = ...
drug = ...
ams = people['AMS']
drug = people['Drug']

In [14]:
_ = ok.grade('q_03_columns')

Run the cell below to assure yourself that the crosstab of the two columns
reconstructs the original counts DataFrame.

In [15]:
# Run this cell.
pd.crosstab(ams, drug)

We are now ready to do the other steps in inference.

First construct a counts table from the null-world, called `fake_ams_counts`.
This is the cross-tabulation of a shuffled version of `ams` with the original
`drug`.


In [16]:
fake_ams_counts = ...
fake_drug = rng.permutation(drug)
fake_ams_counts = pd.crosstab(ams, fake_drug)
# Show the result
fake_ams_counts

In [17]:
_ = ok.grade('q_04_fake_ams_counts')

You can extract the *statistic* for this trial in the null-world, by getting
the value we are interested in, like this:

In [18]:
fake_stat = fake_ams_counts.loc['Had AMS', 'Acetazolamide']
fake_stat

Now you have the procedure to do a single trial, put all this into a `for`
loop, to repeat the following 1000 times:

* Make a new `fake_ams_counts` table.
* Extract the `fake_stat` value of interest.
* Store the `fake_stat` value in an array `fake_stats`.

In [19]:
n_iters = 1000
fake_stats = np.zeros(n_iters)
for i in np.arange(n_iters):
    fake_drug = rng.permutation(drug)
    fake_ams_counts = pd.crosstab(ams, fake_drug)
    fake_stat = fake_ams_counts.loc['Had AMS', 'Acetazolamide']
    fake_stats[i] = fake_stat
# Show the first 10 fake_stats values
fake_stats[:10]

In [20]:
_ = ok.grade('q_05_fake_counts')

Plot a histogram of your `fake_stats` values to see how they relate to the
observed value from the real world, above.

In [21]:
#- Plot a histogram of `fake_stats`
plt.hist(fake_stats);

Calculate the estimated probability `p_est` that we would see a value that is
less than or equal to the observed number of 'No AMS' and 'Acetazolamide', in
the null-world you have just simulated.

In [22]:
p_est = np.count_nonzero(fake_stats <= observed_stat) / n_iters
p_est

In [23]:
_ = ok.grade('q_06_p_est')

## Done.

Congratulations, you're done with the assignment!  Be sure to:

- **run all the tests** (the next cell has a shortcut for that).
- **Save and Checkpoint** from the `File` menu.

In [24]:
# For your convenience, you can run this cell to run all the tests at once!
import os
_ = [ok.grade(q[:-3]) for q in os.listdir("tests") if q.startswith('q')]